# Redis LangChain eCommerce Chatbot

In [ ]:
# Install requirements
!pip install -r requirements.txt

In [ ]:
# Download the dataset
!gdown --id 1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj

In [ ]:
import os
import numpy as np
import pandas as pd


os.environ['OPENAI_API_KEY'] = "YOUR OPENAI API KEY"


## Preprocess dataset

In [ ]:
MAX_TEXT_LENGTH=512
NUMBER_PRODUCTS=2500

def auto_truncate(val):
    return val[:MAX_TEXT_LENGTH]

# Load Product data and truncate long text fields
all_prods_df = pd.read_csv("product_data.csv", converters={
    'bullet_point': auto_truncate,
    'item_keywords': auto_truncate,
    'item_name': auto_truncate
})

In [ ]:
all_prods_df['primary_key'] = all_prods_df['item_id'] + '-' + all_prods_df['domain_name']
all_prods_df['item_keywords'].replace('', np.nan, inplace=True)
all_prods_df.dropna(subset=['item_keywords'], inplace=True)
all_prods_df.reset_index(drop=True, inplace=True)

all_prods_df.head()

In [ ]:
# Get the first 1000 products with non-empty item keywords
product_metadata = all_prods_df.head(NUMBER_PRODUCTS).to_dict(orient='index')

In [ ]:
# Check one of the products
product_metadata[5]

In [ ]:
# Create the content that we will embed with OpenAI and use as context
content = [
    'Item Name: ' + v['item_name'] + '. ' +
    'Description: ' + v['bullet_point'] + '. ' +
    'Other Keywords: ' + v['item_keywords']
    for k, v in product_metadata.items()
]

In [ ]:
content[:10]

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis as RedisVectorDB


vector_db = RedisVectorDB.from_texts(
    texts = content,
    metadatas = list(product_metadata.values()),
    embedding = OpenAIEmbeddings(openai_api_key="YOUR OPENAI API KEY"),
    index_name = "products",
    redis_url = "redis://localhost:6379" # assumes you have a redis stack server running on local host
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain

# from langchain.chat_models import ChatOpenAI
# from langchain.prompts.chat import (
#     ChatPromptTemplate,
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate,
# )


# system_template="""You are a friendly, conversational retail shopping assistant. Use the following product names, descriptions, and keywords to help the shopper find what they want.
# It's ok if you don't know the answer.
# ----------------
# {context}"""

# messages = [
#     SystemMessagePromptTemplate.from_template(system_template),
#     HumanMessagePromptTemplate.from_template("{question}")
# ]

# prompt = ChatPromptTemplate.from_messages(messages)


from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate

template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
Or, end the conversation if it seems like it's done.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

condense_question_prompt = PromptTemplate.from_template(template)

template = """You are a friendly, conversational retail shopping assistant. Use the following product names, descriptions, and keywords to help the shopper find what they want.
It's ok if you don't know the answer.
{context}
Question: {question}
Helpful Answer:"""

qa_prompt= PromptTemplate.from_template(template)


llm = OpenAI(temperature=0)
streaming_llm = OpenAI(streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)

question_generator = LLMChain(llm=llm, prompt=condense_question_prompt)
doc_chain = load_qa_chain(streaming_llm, chain_type="stuff", prompt=qa_prompt)

qa = ConversationalRetrievalChain(
    retriever=vector_db.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)



In [ ]:
chat_history = []

question = input("Hi! What are you looking for today?")

while True:
    result = qa(
        {"question": question, "chat_history": chat_history}
    )
    print("\n")
    chat_history.append((result["question"], result["answer"]))
    question = input()

## Questions for Harrison

- Overall this structure seems to work decently. Anything that you'd recommend out of the box to get better performance? (prompt tuning, llm tweaks)
- Looking at ways to try to "end" the conversation once the user has decided what they want to "buy". For example, sometimes the chat history actually gets in the way and thinks that I am talking about something else.
- Is there a better way to present this than using a while loop?
- See the fork where this repo came from. I tried to take what he did and wrap it in the conversational chain. But he was able to add some mid-stream processing on the documents that come back from Redis. Wondering if there's a way to do that here?